In [1]:
import importlib
import logging
import os

import datasets
import tensorflow as tf

import transformer_document_embedding as tde

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

2023-02-14 11:37:10.614055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 11:37:10.700086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-14 11:37:10.700103: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 11:37:10.726545: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-14 11:37:11.294281: W tensorflow/stream_executor/platform/de

In [2]:
tde = importlib.reload(tde)

In [3]:
LOG_DIR = "../results/notebooks/doc2vec_imdb"
task = tde.tasks.IMDBClassification(data_size_limit=100)
model = tde.models.Doc2VecIMDB(log_dir=LOG_DIR, use_dm=False, dbow_kwargs={"epochs": 1})

2023-02-14 11:37:11,988 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow,d400,n5,mc5,s0.001,t12>', 'datetime': '2023-02-14T11:37:11.988907', 'gensim': '4.2.0', 'python': '3.10.8 (main, Nov  1 2022, 14:18:21) [GCC 12.2.0]', 'platform': 'Linux-6.0.11-1-MANJARO-x86_64-with-glibc2.36', 'event': 'created'}
2023-02-14 11:37:12.010163: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-02-14 11:37:12.010194: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: manjaro-pc
2023-02-14 11:37:12.010200: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: manjaro-pc
2023-02-14 11:37:12.010285: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.60.11
2023-02-14 11:37:12.010301: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.60.11
2023-02-14 11:37:12.010306: I tensorflow/st

In [4]:
doc2vec_train = datasets.combine.concatenate_datasets(
    [task.train, task.unsupervised, task.test]
)
doc2vec_train = doc2vec_train.shuffle()
model._doc2vec.train(doc2vec_train)

2023-02-14 11:37:14,473 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
2023-02-14 11:37:14,490 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-6c9a401648fa09b0.arrow
2023-02-14 11:37:16,753 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
2023-02-14 11:37:16,783 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-9ef2174596950ddb.arrow
2023-02-14 11:37:19,022 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a7068

In [12]:
ds = model._cls_head_dataset(task.train, training=False)

In [13]:
label_ds = task.train.to_tf_dataset(1, columns=["label"]).unbatch()
list(label_ds.take(4).as_numpy_iterator())

[0, 0, 0, 0]

In [14]:
ds = tf.data.Dataset.zip((ds, label_ds))

In [15]:
ds = ds.shuffle(50000)

In [16]:
ds = ds.batch(2)

In [10]:
list(ds.take(4).as_numpy_iterator())

[(array([[[ 0.02674744, -0.00799166, -0.00010695, ...,  0.00455922,
           -0.02033987,  0.00345751],
          [ 0.00544163, -0.00366657, -0.00188221, ...,  0.00255587,
           -0.00328184,  0.00125724]],
  
         [[ 0.0089562 , -0.00161418,  0.00055404, ...,  0.00059678,
           -0.00703504,  0.00362431],
          [ 0.04915053, -0.01072214, -0.00062744, ...,  0.00166242,
           -0.03730201,  0.00970875]]], dtype=float32),
  array([0, 0])),
 (array([[[ 0.03567876, -0.00669367,  0.00111443, ...,  0.00302335,
           -0.02754422,  0.00585555],
          [ 0.02401001, -0.00302373,  0.0013994 , ...,  0.00370954,
           -0.01915143,  0.00506008]],
  
         [[ 0.05356168, -0.01278524, -0.00386273, ...,  0.0021304 ,
           -0.04241763,  0.00804629],
          [ 0.00609807, -0.00285026, -0.00354838, ...,  0.00012294,
           -0.00394088, -0.00058952]]], dtype=float32),
  array([0, 0])),
 (array([[[ 3.0236315e-02, -3.9760964e-03, -2.5477372e-03, ...,
        

In [17]:
ds = model._cls_head_dataset(task.train)